## ascii prefixes

files in the aers zip. (this is from the bundled doc. you can download quarterly data [here](https://fis.fda.gov/extensions/FPD-QDE-FAERS/FPD-QDE-FAERS.html))

1. **DEMO**: patient demographic and administrative information, a single record for each event report.
2. **DRUG**: drug/biologic information for as many medications as were reported for the event (1 or more per event).
3. **REAC**: all "Medical Dictionary for Regulatory Activities" (MedDRA) terms coded for the event www.meddramsso.com
4. **OUTC**: patient outcomes for the event (0 or more).
5. **RPSR**: report sources for event (0 or more).
6. **THER**: drug therapy start dates and end dates for the reported drugs (0 or more per drug per event).
7. **INDI**: all "Medical Dictionary for Regulatory Activities" (MedDRA) terms coded for the indications for use (diagnoses)

In [ ]:
import pathlib, collections, itertools, csv
suffix = '22Q4.txt'
path = pathlib.Path('~/Downloads/aers/ASCII').expanduser()

def aers_rows(prefix: str):
    "return namedtuple + iterator"
    rows = csv.reader(open(path / f'{prefix}{suffix}'), delimiter='$')
    Row = collections.namedtuple(f'{prefix}Row', next(rows))
    return Row, itertools.starmap(Row, rows)


In [ ]:
import collections
drug2active = collections.defaultdict(set)
case2drug = collections.defaultdict(list)
_, drug_rows = aers_rows('DRUG')
for row in drug_rows:
    drug2active[row.drugname].add(row.prod_ai)
    case2drug[row.caseid].append(row.drugname)
print(len(drug2active), 'drugs', len(case2drug), 'cases')

In [ ]:
candidates = collections.defaultdict(set)
for drug, active in drug2active.items():
    full = drug.lower() + ' '.join(x.lower() for x in active)
    if ('adderall' in full or 'amphetamine' in full) and not ('methamphetamine' in full):
        candidates[drug].add(full)
print(len(candidates), 'candidates')

cases = {
    case_id
    for case_id, drugs in case2drug.items()
    if any(x in candidates for x in drugs)
}
print(len(cases), 'cases')

## outcome codes

(from ASC_NTS.pdf in the zip file)

CODE | MEANING_TEXT
---|---
DE | Death
LT | Life-Threatening
HO | Hospitalization - Initial or Prolonged
DS | Disability
CA | Congenital Anomaly
RI | Required Intervention to Prevent Permanent Impairment/Damage
OT | Other Serious (Important Medical Event)


In [ ]:
_ReacRow, reactions = aers_rows('REAC')
case2reac = collections.defaultdict(list)
for row in reactions:
    if row.caseid in cases:
        case2reac[row.caseid].append(row.pt)
reac_counts = collections.Counter(x for val in case2reac.values() for x in val)
print('reactions', len(reac_counts), reac_counts.most_common(100))

case2outc = collections.defaultdict(list)
_OutcRow, outcomes = aers_rows('OUTC')
serious_case = set()
# print(OutcRow._fields)
for row in outcomes:
    if row.caseid in cases:
        case2outc[row.caseid].append(row.outc_cod)
        if row.outc_cod == 'DE':
            serious_case.add(row.caseid)
print('outcomes', collections.Counter(x for val in case2outc.values() for x in val))
print('serious cases', len(serious_case))


In [ ]:
serious_morph = set()
serious_meth = set()
serious_other = set()
for case in serious_case:
    full = ' '.join(case2drug[case])
    if 'ORPH' in full or 'FENT' in full or 'HEROI' in full or 'OXY' in full or 'CODONE' in full:
        serious_morph.add(case)
    elif 'METH' in full or 'COCAI' in full:
        serious_meth.add(case)
    else:
        serious_other.add(case)

print('opioid', len(serious_morph), 'meth-coke', len(serious_meth), 'other', len(serious_other))
print('ndrugs in serious_other', collections.Counter(len(case2drug[case]) for case in serious_other))
for case in serious_other:
    print(case, case2outc[case], case2drug[case])